In [ ]:
import numpy as np
from datetime import datetime
from scipy.interpolate import griddata

from halem import BaseRoadmap, HALEM_space, HALEM_time, plot_timeseries, HALEM_cost,HALEM_co2

import matplotlib.pyplot as plt

plt.style.use("ggplot")
%matplotlib inline

In [ ]:
class RoadMap(BaseRoadmap):
    def load(self):
        d = datetime.strptime("23/03/2019 00:00:00", "%d/%m/%Y %H:%M:%S")
        t0 = d.timestamp()
        x = np.arange(0, 1, 0.04)
        y = np.arange(0, 1, 0.04)
        t = np.arange(t0, (t0 + 2 * 30 * 60 * 30), 30 * 60)

        y, x = np.meshgrid(y, x)
        y = y.reshape(y.size)
        x = x.reshape(y.size)

        nodes = np.zeros((len(x), 2))
        nodes[:, 0] = y
        nodes[:, 1] = x

        u = []
        v = []
        for node in nodes:
            ut = 0 * t + 2 * np.cos(np.pi * (node[0]))
            vt = 0 * t - 2 * np.cos(np.pi * (node[1]))
            u.append(ut)
            v.append(vt)

        v = np.transpose(np.array(v))
        u = np.transpose(np.array(u))
        wd = u * 0 + 20

        return {
            "v": v,
            "u": u,
            "water_depth": wd,
            "time": t,
            "nodes": nodes,
        }

    @staticmethod
    def compute_cost(travel_time, speed):
        week_rate = 700_000 
        fuel_rate = 0.008
        second_rate = week_rate / 7 / 24 / 60 / 60
        return (
            travel_time * second_rate + fuel_rate * travel_time * speed**3
        )
    
    @staticmethod
    def compute_co2(travel_time, speed):
        fuel_rate = 1
        return (fuel_rate * travel_time * speed**3)

In [ ]:
%%time

roadmap = RoadMap(
    number_of_neighbor_layers=2,
    vship=np.array([[1, 4, 8], [1, 5, 10]]),
    WD_min=np.array([8, 7]),
    WVPI=np.array([4000, 10000]),
    WWL=40,
    LWL=80,
    ukc=1.5,
    nl=(3, 2.5),
    dx_min=0.1,
    blend=1,
)

In [ ]:
%%time

roadmap.parse()

In [ ]:
%%time

t0 = "23/03/2019 03:00:00"  # Departure time

start = (0.2, 0.2)
stop = (0.8, 0.8)
v_max = 7


path_t, time_t, dist_t = HALEM_time(start, stop, t0, v_max, roadmap)
path_s, time_s, dist_s = HALEM_space(start, stop, t0, v_max, roadmap)
path_c, time_c, dist_c = HALEM_cost(start, stop, t0, v_max, roadmap)
path_p, time_p, dist_p = HALEM_co2(start, stop, t0, v_max, roadmap)

In [ ]:
fig = plt.figure(figsize=(23, 17))

ax = plt.subplot(3, 2, 1)
plt.axis("square")
a = 1

x_r = np.arange(0, 1, 0.075)
y_r = np.arange(0, 1, 0.075)
y_r, x_r = np.meshgrid(y_r, x_r)

WD_r = griddata(
    (roadmap.nodes[:, 1], roadmap.nodes[:, 0]),
    roadmap.WD[:, 0],
    (x_r, y_r),
    method="linear",
)
u_r = griddata(
    (roadmap.nodes[:, 1], roadmap.nodes[:, 0]),
    roadmap.u[:, 0],
    (x_r, y_r),
    method="linear",
)
v_r = griddata(
    (roadmap.nodes[:, 1], roadmap.nodes[:, 0]),
    roadmap.v[:, 0],
    (x_r, y_r),
    method="linear",
)

cval = np.arange(0, 21, 0.5)
im = plt.contourf(x_r, y_r, WD_r, cval)
fig.colorbar(im, ax=ax, label="Waterdepth in meters")

plt.quiver(
    x_r[::a, ::a], y_r[::a, ::a], u_r[::a, ::a], v_r[::a, ::a], label="flow directions"
)
plt.plot(path_t[:, 0], path_t[:, 1], "r", label="fastest Route", linewidth=3)
plt.plot(path_s[:, 0], path_s[:, 1], "b", label="shortest Route", linewidth=3)
plt.plot(path_c[:, 0], path_c[:, 1], "g", label="cheapest Route", linewidth=3)
plt.plot(path_p[:, 0], path_p[:, 1], "m", label="cleanest Route", linewidth=3)

plt.plot(
    roadmap.nodes[:, 1],
    roadmap.nodes[:, 0],
    "k.",
    label="Nodes of the graph",
    markersize=1,
)
plt.plot(path_t[0, 0], path_t[0, 1], "go", label="start")
plt.plot(path_t[-1, 0], path_t[-1, 1], "ro", label="target")


plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title(r"x/y diagram of the simple rotating flow")
ax.legend(
    loc="upper center", bbox_to_anchor=(-0.5, 0.5), ncol=1, fancybox=True, shadow=True
)
plt.xlabel("lat")
plt.ylabel("lon")

plt.subplot(3, 2, 2)
plot_timeseries(path_t, time_t, roadmap, "r")
plt.title("s/t diagram of the fastest route")


plt.subplot(3, 2, 3)
plot_timeseries(path_s, time_s, roadmap, "b")
plt.title("s/t diagram of the shortest route")


plt.subplot(3, 2, 4)
plot_timeseries(path_c, time_c, roadmap, "g")
plt.title("s/t diagram of the cheapest route")

plt.subplot(3, 2, 5)
plot_timeseries(path_p, time_p, roadmap, "m")
plt.title("s/t diagram of the cleanest route")

plt.show()